<a href="https://colab.research.google.com/github/abcdjdj/cs-766-project/blob/main/double_u_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/'
!unzip -o cvc-final.zip -d /content/
%cd '/content/'

Streaming output truncated to the last 5000 lines.
  inflating: /content/out/mask/479_3.png  
  inflating: /content/out/mask/479_4.png  
  inflating: /content/out/mask/479_5.png  
  inflating: /content/out/mask/479_6.png  
  inflating: /content/out/mask/479_7.png  
  inflating: /content/out/mask/479_8.png  
  inflating: /content/out/mask/479_9.png  
  inflating: /content/out/mask/47_1.png  
  inflating: /content/out/mask/47_10.png  
  inflating: /content/out/mask/47_11.png  
  inflating: /content/out/mask/47_12.png  
  inflating: /content/out/mask/47_13.png  
  inflating: /content/out/mask/47_14.png  
  inflating: /content/out/mask/47_15.png  
  inflating: /content/out/mask/47_16.png  
  inflating: /content/out/mask/47_17.png  
  inflating: /content/out/mask/47_18.png  
  inflating: /content/out/mask/47_19.png  
  inflating: /content/out/mask/47_2.png  
  inflating: /content/out/mask/47_20.png  
  inflating: /content/out/mask/47_21.png  
  inflating: /content/out/mask/47_22.png  
  inf

Defining Device

In [ ]:
import torch
# import torch_xla
# import torch_xla.core.xla_model as xm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()

Utility Functions

In [ ]:
import cv2
import glob
import numpy as np
from tqdm import tqdm as tqdm

'''
Reads the image specified by 'path' and returns it
param : path - path of image file
return : image as a numpy array
'''
def read_img(path):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = np.clip(image - np.median(image)+127, 0, 255)
    image = image/255.0
    image = image.astype(np.float32)
    return image

def read_mask(path):
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    mask = mask/255.0
    mask = mask.astype(np.float32)
    #mask = np.expand_dims(mask, axis=-1)
    return mask

'''
Converts numpy img to tensor
param : img - numpy arr containing image data
return : t - torch tensor of shape [1, 3, H, W]
'''
def img_to_tensor(img):
    t = torch.from_numpy(img)
    t = t.view(-1, 3, t.shape[0], t.shape[1])
    return t

def mask_to_tensor(mask):
    t = torch.from_numpy(mask)
    t = t.view(-1, t.shape[0], t.shape[1])
    return t

'''
t - tensor of shape [H, W]
'''
def tensor_to_mask(t):
    t = t.view(t.shape[0], t.shape[1])
    return t.numpy()

'''
Converts tensor back to numpy img
param : t - torch tensor of shape [1, 3, H, W]
return : img - numpy arr containing image data
'''
def tensor_to_img(t):
    t = t.view(t.shape[2], t.shape[3], 3)
    return t.numpy()

Double U-Net Architecture

Wrap Up inside nn.Module

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torchvision import models
from imageio import imread as imread
import matplotlib.pyplot as plt

class SqueezeAndExcite(nn.Module):
  def __init__(self, x, ratio = 8):
    super(SqueezeAndExcite, self).__init__()

    channel_axis = 1
    filters = x.shape[channel_axis]
    # Architecture
    self.avgpool2d = nn.AvgPool2d(kernel_size = (x.shape[2], x.shape[3]))
    self.sequential = nn.Sequential(nn.Linear(filters, filters//ratio, bias = False), nn.ReLU(), nn.Linear(filters//ratio, filters, bias = False), nn.Sigmoid())

  def forward(self, x):
    init = x
    #channel_axis = 1
    filters = init.shape[1]
    x = self.avgpool2d(x)
    x = x.view(init.shape[0] , filters)
    x = self.sequential(x)
    x = x.view(init.shape[0], filters, 1, 1)

    return torch.mul(init, x)

class ConvBlock(nn.Module):
  def __init__(self, x, filters):
      super(ConvBlock, self).__init__()

      self.layer1_conv2d = nn.Conv2d(in_channels = x.shape[1], out_channels = filters, kernel_size = 3, padding='same')
      x = self.layer1_conv2d(x)
      self.layer1_batchnorm2d = nn.BatchNorm2d(num_features = x.shape[1])
      x = self.layer1_batchnorm2d(x)
      self.layer1_relu = nn.ReLU()
      x = self.layer1_relu(x)

      self.layer2_conv2d = nn.Conv2d(in_channels = x.shape[1], out_channels = filters, kernel_size = 3, padding='same')
      x = self.layer2_conv2d(x)
      self.layer2_batchnorm2d = nn.BatchNorm2d(num_features = x.shape[1])
      x = self.layer2_batchnorm2d(x)
      self.layer2_relu = nn.ReLU()
      x = self.layer2_relu(x)

      self.squeeze_and_excite = SqueezeAndExcite(x)

  def forward(self, x):
      x = self.layer1_conv2d(x)
      x = self.layer1_batchnorm2d(x)
      x = self.layer1_relu(x)

      x = self.layer2_conv2d(x)
      x = self.layer2_batchnorm2d(x)
      x = self.layer2_relu(x)

      x = self.squeeze_and_excite.forward(x)
      return x

class ASPP(nn.Module):
    def __init__(self, x, filter_count):
      super(ASPP, self).__init__()

      self.layer1_avgpool2d = nn.AvgPool2d(kernel_size = (x.shape[2], x.shape[3]))
      se = self.layer1_avgpool2d(x)
      self.layer1_conv2d = nn.Conv2d(in_channels = se.shape[1], out_channels = filter_count, kernel_size = 1, padding='same')
      se = self.layer1_conv2d(se)
      self.layer1_batchnorm2d = nn.BatchNorm2d(num_features = se.shape[1])
      se = self.layer1_batchnorm2d(se)
      self.layer1_relu = nn.ReLU()
      se = self.layer1_relu(se)
      self.layer1_upsampling = nn.UpsamplingBilinear2d(size=(x.shape[2], x.shape[3]))
      se = self.layer1_upsampling(se)

      self.layer2_conv2d = nn.Conv2d(dilation=1, in_channels = x.shape[1], out_channels = filter_count, kernel_size = 1, padding='same', bias=False)
      y1 = self.layer2_conv2d(x)
      self.layer2_batchnorm2d = nn.BatchNorm2d(num_features = y1.shape[1])
      y1 = self.layer2_batchnorm2d(y1)
      self.layer2_relu = nn.ReLU()
      y1 = self.layer2_relu(y1)

      self.layer3_conv2d = nn.Conv2d(dilation=6, in_channels = x.shape[1], out_channels = filter_count, kernel_size = 1, padding='same', bias=False)
      y2 = self.layer3_conv2d(x)
      self.layer3_batchnorm2d = nn.BatchNorm2d(num_features = y2.shape[1])
      y2 = self.layer3_batchnorm2d(y2)
      self.layer3_relu = nn.ReLU()
      y2 = self.layer3_relu(y2)

      self.layer4_conv2d = nn.Conv2d(dilation=12, in_channels = x.shape[1], out_channels = filter_count, kernel_size = 1, padding='same', bias=False)
      y3 = self.layer4_conv2d(x)
      self.layer4_batchnorm2d = nn.BatchNorm2d(num_features = y3.shape[1])
      y3 = self.layer4_batchnorm2d(y3)
      self.layer4_relu = nn.ReLU()
      y3 = self.layer4_relu(y3)

      self.layer5_conv2d = nn.Conv2d(dilation=18, in_channels = x.shape[1], out_channels = filter_count, kernel_size = 1, padding='same', bias=False)
      y4 = self.layer5_conv2d(x)
      self.layer5_batchnorm2d = nn.BatchNorm2d(num_features = y4.shape[1])
      y4 = self.layer5_batchnorm2d(y4)
      self.layer5_relu = nn.ReLU()
      y4 = self.layer5_relu(y4)

      y = torch.cat([se, y1, y2, y3, y4], dim=1)
      self.layer6_conv2d = nn.Conv2d(dilation=1, in_channels = y.shape[1], out_channels = filter_count, kernel_size = 1, padding='same', bias=False)
      y = self.layer6_conv2d(y)
      self.layer6_batchnorm2d = nn.BatchNorm2d(num_features = y.shape[1])
      y = self.layer6_batchnorm2d(y)
      self.layer6_relu = nn.ReLU()
      y = self.layer6_relu(y)

    def forward(self, x, filter_count):
      se = self.layer1_avgpool2d(x)
      se = self.layer1_conv2d(se)
      se = self.layer1_batchnorm2d(se)
      se = self.layer1_relu(se)
      se = self.layer1_upsampling(se)
      #print(se.shape)

      y1 = self.layer2_conv2d(x)
      y1 = self.layer2_batchnorm2d(y1)
      y1 = self.layer2_relu(y1)
      #print(y1.shape)

      y2 = self.layer3_conv2d(x)
      y2 = self.layer3_batchnorm2d(y2)
      y2 = self.layer3_relu(y2)
      #print(y2.shape)

      y3 = self.layer4_conv2d(x)
      y3 = self.layer4_batchnorm2d(y3)
      y3 = self.layer4_relu(y3)
      #print(y3.shape)

      y4 = self.layer5_conv2d(x)
      y4 = self.layer5_batchnorm2d(y4)
      y4 = self.layer5_relu(y4)
      #print(y4.shape)

      y = torch.cat([se, y1, y2, y3, y4], dim=1)
      del x, se, y1, y2, y3, y4
      y = self.layer6_conv2d(y)
      y = self.layer6_batchnorm2d(y)
      y = self.layer6_relu(y)
      #print(y.shape)
      return y

class Encoder1(nn.Module):
    def __init__(self):
      super(Encoder1, self).__init__()
      # self.model = models.vgg19(pretrained = True)
      # self.vgg19_final_op = None
    
    def forward(self, inputs):
      #skip connections from pre-trained VGG-19
      names = ["ReLU-4", "ReLU-9", "ReLU-18", "ReLU-27", "ReLU-36"]
      model = models.vgg19(pretrained = True)
      
      if inputs.device == device:
        model = model.to(device)

      indices = [3, 8, 17, 26, 35]

      skip_connections = []

      def encoder1_receive_outputs(layer, _, output):
          skip_connections.append(output.detach())
          

      for name, layer in model.named_children():
          for idx in indices:
              layer[idx].register_forward_hook(encoder1_receive_outputs)
          break

      #self.model(inputs)
      model(inputs)

      # print(f'[Encoder1] Op size = {skip_connections[-1].shape}')
      # print(f'[Encoder1] 0 size = {skip_connections[0].shape}')
      # print(f'[Encoder1] 1 size = {skip_connections[1].shape}')
      # print(f'[Encoder1] 2 size = {skip_connections[2].shape}')
      # print(f'[Encoder1] 3 size = {skip_connections[3].shape}')
      return skip_connections[-1], skip_connections[0:-1]
      # op = torch.ones(3, 512, 18, 24)
      # skip_connections = [torch.ones(3, 64, 288, 384), torch.ones(3, 128, 144, 192), torch.ones(3, 256, 72, 96), torch.ones(3, 512, 36, 48)]
      # return op, skip_connections

class Decoder1(nn.Module):
    def __init__(self, inputs, skip_connections):
        super(Decoder1, self).__init__()
        num_filters = [256, 128, 64, 32]
        skip_connections.reverse()
        x = inputs

        self.layer1_upsampling = nn.UpsamplingBilinear2d(size=(2*x.shape[2], 2*x.shape[3]))
        x = self.layer1_upsampling(x)
        x = torch.cat([x, skip_connections[0]], dim=1)
        self.layer1_convblock = ConvBlock(x, num_filters[0])
        x = self.layer1_convblock.forward(x)

        self.layer2_upsampling = nn.UpsamplingBilinear2d(size=(2*x.shape[2], 2*x.shape[3]))
        x = self.layer2_upsampling(x)
        x = torch.cat([x, skip_connections[1]], dim=1)
        self.layer2_convblock = ConvBlock(x, num_filters[1])
        x = self.layer2_convblock.forward(x)

        self.layer3_upsampling = nn.UpsamplingBilinear2d(size=(2*x.shape[2], 2*x.shape[3]))
        x = self.layer3_upsampling(x)
        x = torch.cat([x, skip_connections[2]], dim=1)
        self.layer3_convblock = ConvBlock(x, num_filters[2])
        x = self.layer3_convblock.forward(x)

        self.layer4_upsampling = nn.UpsamplingBilinear2d(size=(2*x.shape[2], 2*x.shape[3]))
        x = self.layer4_upsampling(x)
        x = torch.cat([x, skip_connections[3]], dim=1)
        self.layer4_convblock = ConvBlock(x, num_filters[3])
        x = self.layer4_convblock.forward(x)

        # Undo the reversal so that forward passes don't get screwed
        skip_connections.reverse()
    
    def forward(self, x, skip_connections):
        num_filters = [256, 128, 64, 32]
        skip_connections.reverse()

        x = self.layer1_upsampling(x)
        x = torch.cat([x, skip_connections[0]], dim=1)
        x = self.layer1_convblock.forward(x)

        x = self.layer2_upsampling(x)
        x = torch.cat([x, skip_connections[1]], dim=1)
        x = self.layer2_convblock.forward(x)

        x = self.layer3_upsampling(x)
        x = torch.cat([x, skip_connections[2]], dim=1)
        x = self.layer3_convblock.forward(x)

        x = self.layer4_upsampling(x)
        x = torch.cat([x, skip_connections[3]], dim=1)
        x = self.layer4_convblock.forward(x)

        del skip_connections

        return x

class Encoder2(nn.Module):
    def __init__(self, inputs):
        super(Encoder2, self).__init__()
        num_filters = [32, 64, 128, 256]
        x = inputs

        self.layer1_convblock = ConvBlock(x, num_filters[0])
        x = self.layer1_convblock.forward(x)
        self.layer1_maxpool2d = nn.MaxPool2d(kernel_size = (2,2))
        x = self.layer1_maxpool2d(x)

        self.layer2_convblock = ConvBlock(x, num_filters[1])
        x = self.layer2_convblock.forward(x)
        self.layer2_maxpool2d = nn.MaxPool2d(kernel_size = (2,2))
        x = self.layer2_maxpool2d(x)

        self.layer3_convblock = ConvBlock(x, num_filters[2])
        x = self.layer3_convblock.forward(x)
        self.layer3_maxpool2d = nn.MaxPool2d(kernel_size = (2,2))
        x = self.layer3_maxpool2d(x)

        self.layer4_convblock = ConvBlock(x, num_filters[3])
        x = self.layer4_convblock.forward(x)
        self.layer4_maxpool2d = nn.MaxPool2d(kernel_size = (2,2))
        x = self.layer4_maxpool2d(x)
    
    def forward(self, x):
        num_filters = [32, 64, 128, 256]
        skip_connections = []

        x = self.layer1_convblock.forward(x)
        skip_connections.append(x)
        x = self.layer1_maxpool2d(x)

        x = self.layer2_convblock.forward(x)
        skip_connections.append(x)
        x = self.layer2_maxpool2d(x)

        x = self.layer3_convblock.forward(x)
        skip_connections.append(x)
        x = self.layer3_maxpool2d(x)

        x = self.layer4_convblock.forward(x)
        skip_connections.append(x)
        x = self.layer4_maxpool2d(x)

        return x, skip_connections

class Decoder2(nn.Module):
      def __init__(self, inputs, skip_1, skip_2):
          super(Decoder2, self).__init__()
          num_filters = [256, 128, 64, 32]

          skip_2.reverse()
          x = inputs

          self.layer1_upsampling = nn.UpsamplingBilinear2d(size=(2*x.shape[2], 2*x.shape[3]))
          x = self.layer1_upsampling(x)
          x = torch.cat([x, skip_1[0], skip_2[0]], dim=1)
          self.layer1_convblock = ConvBlock(x, num_filters[0])
          x = self.layer1_convblock.forward(x)

          self.layer2_upsampling = nn.UpsamplingBilinear2d(size=(2*x.shape[2], 2*x.shape[3]))
          x = self.layer2_upsampling(x)
          x = torch.cat([x, skip_1[1], skip_2[1]], dim=1)
          self.layer2_convblock = ConvBlock(x, num_filters[1])
          x = self.layer2_convblock.forward(x)

          self.layer3_upsampling = nn.UpsamplingBilinear2d(size=(2*x.shape[2], 2*x.shape[3]))
          x = self.layer3_upsampling(x)
          x = torch.cat([x, skip_1[2], skip_2[2]], dim=1)
          self.layer3_convblock = ConvBlock(x, num_filters[2])
          x = self.layer3_convblock.forward(x)

          self.layer4_upsampling = nn.UpsamplingBilinear2d(size=(2*x.shape[2], 2*x.shape[3]))
          x = self.layer4_upsampling(x)
          x = torch.cat([x, skip_1[3], skip_2[3]], dim=1)
          self.layer4_convblock = ConvBlock(x, num_filters[3])
          x = self.layer4_convblock.forward(x)

          skip_2.reverse() # Undo the reverse so we don't screw up forward()
      
      def forward(self, x, skip_1, skip_2):
          num_filters = [256, 128, 64, 32]

          skip_2.reverse()

          x = self.layer1_upsampling(x)
          x = torch.cat([x, skip_1[0], skip_2[0]], dim=1)
          x = self.layer1_convblock.forward(x)

          x = self.layer2_upsampling(x)
          x = torch.cat([x, skip_1[1], skip_2[1]], dim=1)
          x = self.layer2_convblock.forward(x)

          x = self.layer3_upsampling(x)
          x = torch.cat([x, skip_1[2], skip_2[2]], dim=1)
          x = self.layer3_convblock.forward(x)

          x = self.layer4_upsampling(x)
          x = torch.cat([x, skip_1[3], skip_2[3]], dim=1)
          x = self.layer4_convblock.forward(x)

          del skip_1, skip_2

          return x

class OutputBlock(nn.Module):
      def __init__(self, inputs):
          super(OutputBlock, self).__init__()
          self.conv2d = nn.Conv2d(in_channels = inputs.shape[1], out_channels = 1, kernel_size = 1, padding = "same")
          self.sigmoid = nn.Sigmoid()
      
      def forward(self, x):
          x = self.conv2d(x)
          x = self.sigmoid(x)
          return x

class DoubleUNet(nn.Module):
  def __init__(self, inputs):
      with torch.no_grad():
        super(DoubleUNet, self).__init__()

        # Encoder 1
        self.encoder1 = Encoder1()
        encoder1_op, encoder1_skip_conns = self.encoder1.forward(inputs)

        # ASPP
        self.aspp1 = ASPP(encoder1_op, 64)
        aspp_op = self.aspp1.forward(encoder1_op, 64)

        # Decoder 1
        self.decoder1 = Decoder1(aspp_op, encoder1_skip_conns)
        decoder1_op = self.decoder1.forward(aspp_op, encoder1_skip_conns)

        # Output 1
        self.outputblock1 = OutputBlock(decoder1_op)
        mask = self.outputblock1.forward(decoder1_op)
        network1_op = inputs * mask

        # Encoder 2
        self.encoder2 = Encoder2(network1_op)
        encoder2_op,encoder2_skip_conns = self.encoder2.forward(network1_op)

        # ASPP 2
        self.aspp2 = ASPP(encoder2_op, 64)
        aspp2_op = self.aspp2.forward(encoder2_op, 64)

        # Decoder 2
        self.decoder2 = Decoder2(aspp2_op, encoder1_skip_conns, encoder2_skip_conns)
        decoder2_op = self.decoder2.forward(aspp2_op, encoder1_skip_conns, encoder2_skip_conns)

        # Output 2
        self.outputblock2 = OutputBlock(decoder2_op)

  
  def forward(self, inputs):
      # Encoder 1
      encoder1_op, encoder1_skip_conns = self.encoder1.forward(inputs)
      encoder1_op = encoder1_op.to(device)
      encoder1_skip_conns[0] = encoder1_skip_conns[0].to(device)
      encoder1_skip_conns[1] = encoder1_skip_conns[1].to(device)
      encoder1_skip_conns[2] = encoder1_skip_conns[2].to(device)
      encoder1_skip_conns[3] = encoder1_skip_conns[3].to(device)

      # ASPP
      aspp_op = self.aspp1.forward(encoder1_op, 64)
      del encoder1_op

      # Decoder 1
      decoder1_op = self.decoder1.forward(aspp_op, encoder1_skip_conns)
      del aspp_op

      # Output 1
      mask = self.outputblock1.forward(decoder1_op)
      network1_op = inputs * mask

      # Encoder 2
      encoder2_op,encoder2_skip_conns = self.encoder2.forward(network1_op)
      del network1_op

      # ASPP 2
      aspp2_op = self.aspp2.forward(encoder2_op, 64)
      del encoder2_op

      # Decoder 2
      decoder2_op = self.decoder2.forward(aspp2_op, encoder1_skip_conns, encoder2_skip_conns)
      del aspp2_op, encoder1_skip_conns, encoder2_skip_conns

      # Output 2
      network2_op = self.outputblock2.forward(decoder2_op)

      final_output = torch.cat([mask, network2_op], dim = 1)
      del mask, network2_op
      return final_output

Data Pre-Processing

In [ ]:
img_list = [] #sorted(glob.glob("out/image/*"))
mask_list = [] #sorted(glob.glob("out/mask/*"))

for i in range(1, 5):
    img_list += sorted(glob.glob("out/image/*_" + str(i) + ".png"))
    mask_list += sorted(glob.glob("out/mask/*_" + str(i) + ".png"))

print(len(img_list))
print(len(mask_list))

2448
2448


In [ ]:
#img_list = [img_to_tensor(read_img(ele)) for ele in img_list]
#mask_list = [img_to_tensor(read_img(ele)) for ele in mask_list]

img_data = list(zip(img_list,mask_list))

data_len = len(img_data)

In [ ]:
#Hyperparameters

learning_rate = 1e-4
num_epochs = 50
batch_size = 14

In [ ]:
import math
#Splitting into 80-10-10

train_set, val_set, test_set = torch.utils.data.random_split(img_data, [round(0.8*data_len), round(0.1*data_len), data_len - round(0.8*data_len) - round(0.1*data_len)])

num_batches = math.ceil(len(train_set)/batch_size)
print(f'Train Set = {len(train_set)}')
print(f'Val Set = {len(val_set)}')
print(f'Test Set = {len(test_set)}')

Train Set = 1958
Val Set = 245
Test Set = 245


In [ ]:
#Divide Train Data Into List of Batches for Training Loop
train_loader_x = []
train_loader_y = []

for idx in range(0, len(train_set), batch_size):
  if idx + batch_size > len(train_set):
    x_tup, y_tup = list(zip(*(list(train_set)[idx:])))
  else:
    x_tup, y_tup = list(zip(*(list(train_set)[idx:idx + batch_size])))
  train_loader_x.append(x_tup)
  train_loader_y.append(y_tup)

# print(f'Len = {len(train_loader_x[-1])}')
# print(train_loader_x[-1])
#print(train_loader_x) #is a list of tuples, in which each tuple is batch_size length
#print(train_loader_y)

Define Optimizer, Loss Function

In [ ]:
tmp = torch.ones(batch_size, 3, 288, 384)
print(torch.cuda.memory_allocated() / (1024 * 1024))
with torch.no_grad():
  double_u_net = DoubleUNet(tmp).to(device)
print(torch.cuda.memory_allocated() / (1024 * 1024))
# for parameter in double_u_net.parameters():
#    print(f"Parameter = {parameter}")

optimizer = optim.NAdam(double_u_net.parameters(), lr = 0.001)

criterion  = nn.BCELoss()

!nvidia-smi

0.0
30.92431640625
Wed Apr 20 19:30:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    29W /  70W |   1368MiB / 15109MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                     

Store Losses

In [ ]:
losses = []

Training Loop

In [ ]:
# img_data = torch.ones(batch_size, 3, 288, 384).to(device)
# mask_data = torch.ones(batch_size, 2, 288, 384).to(device)

for epochs in tqdm(range(num_epochs)):
    running_loss = 0
    for idx in tqdm(range(num_batches)):
        img_data = [img_to_tensor(read_img(ele)) for ele in train_loader_x[idx]]
        img_data = torch.cat(img_data, dim = 0).to(device)
        mask_data = [mask_to_tensor(read_mask(ele)).repeat(2, 1, 1) for ele in train_loader_y[idx]]
        mask_data = torch.stack(mask_data, dim = 0).to(device)

        # print('Before forward pass ->')
        # print(torch.cuda.memory_allocated() / (1024 * 1024))
        
        mask_pred = double_u_net.forward(img_data.float())

        # print('After forward pass ->')
        # print(torch.cuda.memory_allocated() / (1024 * 1024))

        del img_data
        loss = criterion(mask_pred, mask_data)
        del mask_data, mask_pred
        optimizer.zero_grad()
        loss.backward()

        # print('After backward pass ->')
        # print(torch.cuda.memory_allocated() / (1024 * 1024))

        optimizer.step()
        running_loss += float(loss.detach())
        del loss
        #losses.append(running_loss)
        #torch.cuda.empty_cache()
        #!nvidia-smi

        print('*************')

    print(f"For epoch {epochs + 1}, BCE Loss is {running_loss}")
    if (epochs + 1) % 5 == 0:
        torch.save(double_u_net.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/double_unet_cvc-clinic_{epochs+1}_{running_loss}.pt')

# Save PyTorch model to disk
torch.save(double_u_net.state_dict(), '/content/drive/MyDrive/Colab Notebooks/double_unet_cvc-clinic.pt')

  1%|          | 1/140 [00:03<08:45,  3.78s/it]

*************



  1%|▏         | 2/140 [00:06<07:52,  3.42s/it]

*************



  2%|▏         | 3/140 [00:10<07:32,  3.30s/it]

...

*************



 29%|██▉       | 41/140 [02:16<05:28,  3.31s/it]

*************



 30%|███       | 42/140 [02:20<05:24,  3.31s/it]

*************


In [ ]:
# validation_data = [img_to_tensor(read_img(ele[0])) for ele in val_set]
# #validation_data = [img_to_tensor(read_img(ele)) for ele in [r'out/image/1_12.png', r'out/image/1_4.png', r'out/image/1_6.png']]
# validation_data = torch.cat(validation_data, dim = 0).to(device)
# validation_data = validation_data.float()
# print(validation_data.shape)

# validation_mask = double_u_net.forward(validation_data).detach().cpu()

In [ ]:
# from matplotlib import pyplot as plt

# mask_inference = tensor_to_mask(validation_mask[2][0])
# print(mask_inference.shape)
# for ele in val_set:
#     print(ele[0])

# #Show the image with matplotlib
# plt.imshow(mask_inference, cmap='Greys_r')
# plt.show()


# import time
# start = time.process_time()
# model = models.vgg19(pretrained = True)
# model = model.to(device)
# print(time.process_time() - start)

In [ ]:
# t0 = torch.randn(2, 2).to(device)#creating a tensor and sending it to the TPU
# t1 = torch.randn(2, 2).to(device)#creating a tensor and sending it to the TPU
# print(t0 + t1) # As both tensors are now on the same device  i.e same TPU core we can perform any calculation on them like addition
# print(t0.mm(t1)) # Matrix Multiplication

In [ ]:
# if t0.device == device:
#   print("yes")